<a href="https://colab.research.google.com/github/VesalAhsani/Driver-behavior-detection/blob/main/DMS_YOLOv8m_CLS_224_fullAugmented_focal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ Install compatible versions
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install ultralytics
!pip install numpy==2.0.0 scipy==1.14.1 pandas==2.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 154.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 153.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 130.5 MB/s eta 0:00:00
     ━━━━━━━

In [2]:
import torch
import ultralytics
import pandas as pd

print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("YOLOv8 Version:", ultralytics.__version__)
print("Pandas Version:", pd.__version__)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
PyTorch Version: 2.5.1+cu121
CUDA Available: True
YOLOv8 Version: 8.3.107
Pandas Version: 2.2.2


In [3]:
# ✅ **Step 1: Verify GPU is available**
import torch
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Ensure using GPU 0 (adjust if needed)

print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())
print("CUDA Current Device:", torch.cuda.current_device() if torch.cuda.is_available() else "No CUDA device")
print("CUDA Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA device")

# ✅ **Step 2: Mount Google Drive**
from google.colab import drive
drive.mount('/content/drive')

# ✅ **Step 3: Unzipping Dataset**
import zipfile

dataset_path = '/content/drive/MyDrive/Split_mixed_newaugmented_224.zip'
extract_path = '/content/Split_mixed_newaugmented_224'

if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"❌ The dataset file at {dataset_path} was not found!")

print("✅ Dataset zip file exists. Unzipping...")

with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset unzipped successfully!")

# ✅ **Step 4: Verify dataset paths**
dataset_actual_path = os.path.join(extract_path, "Split_224")
if not os.path.exists(os.path.join(dataset_actual_path, "train")):
    raise FileNotFoundError("❌ Train folder not found in dataset path!")

# ✅ **Step 5: Install YOLOv8**
!pip install ultralytics --upgrade

# ✅ **Step 6: Import YOLO and Torch**
from ultralytics import YOLO
import torch.nn as nn
import torch.nn.functional as F

print("✅ Ultralytics library imported successfully!")

# ✅ **Step 7: Verify dataset structure**
from ultralytics.data.utils import check_cls_dataset
dataset_info = check_cls_dataset(dataset_actual_path)
print(f"✅ Dataset verified: {dataset_info}")

# ✅ **Step 8: Count Training and Validation Images**
import glob

train_images = glob.glob(os.path.join(dataset_actual_path, "train", "*", "*.jpg"))
val_images = glob.glob(os.path.join(dataset_actual_path, "val", "*", "*.jpg"))

if not train_images or not val_images:
    raise ValueError("❌ Training or validation images were not found. Check dataset paths.")
print(f"✅ Number of training images: {len(train_images)}")
print(f"✅ Number of validation images: {len(val_images)}")

# ✅ **Step 9: Define Dataset Path**
DATASET_PATH = dataset_actual_path
OUTPUT_PATH = '/content/drive/MyDrive/yolov8m_cls_results_focal_224'

# ✅ **Step 10: Compute Alpha Weights for Focal Loss**
class_counts = {
    "Control_Panel": 11700,
    "Drinking": 11700,
    "Eating": 2689,
    "Makeup": 11700,
    "Normal": 12196,
    "Phone_Call_(Left)": 11700,
    "Phone_Call_(Right)": 11700,
    "Reaching_Behind": 11700,
    "Sleep": 12232,
    "Smoking": 12376,
    "Talk_to_passengers": 11700,
    "Text_(Left)": 11700,
    "Text_(Right)": 11700,
    "Yawning": 11700,
}

max_samples = max(class_counts.values())
alpha = torch.tensor([max_samples / class_counts[class_name] for class_name in class_counts], dtype=torch.float32).to("cuda")

print(f"✅ Calculated Alpha Weights: {alpha}")

# ✅ **Step 11: Define Focal Loss**
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        log_probs = F.log_softmax(inputs, dim=-1)
        probs = torch.exp(log_probs)

        target_probs = probs.gather(dim=-1, index=targets.unsqueeze(-1)).squeeze(-1)
        focal_factor = (1 - target_probs) ** self.gamma
        loss = -focal_factor * log_probs.gather(dim=-1, index=targets.unsqueeze(-1)).squeeze(-1)

        if self.alpha is not None:
            alpha_factor = self.alpha.gather(dim=-1, index=targets.unsqueeze(-1)).squeeze(-1)
            loss *= alpha_factor

        return loss.mean() if self.reduction == 'mean' else loss.sum()

# ✅ **Step 12: Initialize YOLOv8 Model**
model = YOLO('yolov8m-cls.pt')

# ✅ **Step 13: Attach Focal Loss**
model.loss = FocalLoss(gamma=2.0, alpha=alpha, reduction='mean')


# ✅ **Step 15: Train the Model**
model.train(
    data=DATASET_PATH,
    epochs=50,
    batch=32,
    imgsz=224,
    project=OUTPUT_PATH,
    name='yolov8m_cls_results_focal_224',
    pretrained=True,
    optimizer='Adam',
    lr0=0.0005,
    resume=False,
)
print("✅ Training completed!")

# ✅ **Step 16: Evaluate the Model**
results = model.val()
print("✅ Validation completed!")

# ✅ **Step 17: Save Best Weights**
best_weights_path = f"{OUTPUT_PATH}/yolov8m_cls_results_focal_224/weights/best.pt"
os.makedirs(f"{OUTPUT_PATH}/yolov8m_cls_results_focal_224/weights", exist_ok=True)
!cp /content/runs/classify/yolov8m_cls_results_focal_224/weights/best.pt {best_weights_path}
print(f"Best weights saved to {best_weights_path}")


CUDA Available: True
CUDA Device Count: 1
CUDA Current Device: 0
CUDA Device Name: NVIDIA A100-SXM4-40GB
Mounted at /content/drive
✅ Dataset zip file exists. Unzipping...
✅ Dataset unzipped successfully!
✅ Ultralytics library imported successfully!
train: /content/Split_mixed_newaugmented_224/Split_224/train... found 156504 images in 14 classes ✅ 
val: /content/Split_mixed_newaugmented_224/Split_224/val... found 17276 images in 14 classes ✅ 
test: None...
✅ Dataset verified: {'train': PosixPath('/content/Split_mixed_newaugmented_224/Split_224/train'), 'val': PosixPath('/content/Split_mixed_newaugmented_224/Split_224/val'), 'test': None, 'nc': 14, 'names': {0: 'Control_Panel', 1: 'Drinking', 2: 'Eating', 3: 'Makeup', 4: 'Normal', 5: 'Phone_Call_(Left)', 6: 'Phone_Call_(Right)', 7: 'Reaching_Behind', 8: 'Sleep', 9: 'Smoking', 10: 'Talk_to_passengers', 11: 'Text_(Left)', 12: 'Text_(Right)', 13: 'Yawning'}}
✅ Number of training images: 156504
✅ Number of validation images: 17276
✅ Calculat

100%|██████████| 32.7M/32.7M [00:00<00:00, 216MB/s]


Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=/content/Split_mixed_newaugmented_224/Split_224, epochs=50, time=None, patience=100, batch=32, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8m_cls_results_focal_224, name=yolov8m_cls_results_focal_224, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, 

100%|██████████| 5.35M/5.35M [00:00<00:00, 66.1MB/s]


AMP: checks passed ✅


...


50 epochs completed in 3.106 hours.
Optimizer stripped from /content/drive/MyDrive/yolov8m_cls_results_focal_224/yolov8m_cls_results_focal_224/weights/last.pt, 31.7MB
Optimizer stripped from /content/drive/MyDrive/yolov8m_cls_results_focal_224/yolov8m_cls_results_focal_224/weights/best.pt, 31.7MB

Validating /content/drive/MyDrive/yolov8m_cls_results_focal_224/yolov8m_cls_results_focal_224/weights/best.pt...
Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv8m-cls summary (fused): 42 layers, 15,780,590 parameters, 0 gradients, 41.6 GFLOPs
train: /content/Split_mixed_newaugmented_224/Split_224/train... found 156504 images in 14 classes ✅ 
val: /content/Split_mixed_newaugmented_224/Split_224/val... found 17276 images in 14 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 270/270 [00:06<00:00, 42.19it/s]


                   all      0.992          1
Speed: 0.1ms preprocess, 0.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/yolov8m_cls_results_focal_224/yolov8m_cls_results_focal_224
✅ Training completed!
Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv8m-cls summary (fused): 42 layers, 15,780,590 parameters, 0 gradients, 41.6 GFLOPs
train: /content/Split_mixed_newaugmented_224/Split_224/train... found 156504 images in 14 classes ✅ 
val: /content/Split_mixed_newaugmented_224/Split_224/val... found 17276 images in 14 classes ✅ 
test: None...


val: Scanning /content/Split_mixed_newaugmented_224/Split_224/val... 17276 images, 0 corrupt: 100%|██████████| 17276/17276 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 540/540 [00:07<00:00, 72.31it/s]


                   all      0.992          1
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/yolov8m_cls_results_focal_224/yolov8m_cls_results_focal_2242
✅ Validation completed!
cp: cannot stat '/content/runs/classify/yolov8m_cls_results_focal_224/weights/best.pt': No such file or directory
Best weights saved to /content/drive/MyDrive/yolov8m_cls_results_focal_224/yolov8m_cls_results_focal_224/weights/best.pt
